In [1]:
from src.XMLUtils import DocumentCreator

In [2]:
document_name = "Levels_of_AGI_Operationalizing_Progress_on_the_Path_to_AGI"
base_path = "/home/paper_translator/data"
document_path = f"{base_path}/documents/{document_name}"
xml_path = f"{document_path}/{document_name}.tei.xml"

In [13]:
creator = DocumentCreator()
creator.load_xml(xml_path, contain_abst=False)
documents = creator.create_docs()
documents

[Document(id_='0', embedding=None, metadata={'Section No.': '3.', 'Section Title': 'Focus on Cognitive and Metacognitive Tasks.', 'Title': 'Levels of AGI: Operationalizing Progress on the Path to AGI', 'Authors': 'Meredith Ringel Morris, Jascha Sohl-Dickstein, Noah Fiedel, Tris Warkentin, Allan Dafoe, Aleksandra Faust, Clement Farabet, Shane Legg, Google Deepmind', 'Idno': 'arXiv:2311.02462v1[cs.AI]', 'Published': '4 Nov 2023', 'Language': 'en'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, hash='7ac2cb246de401253da2ee4c765bf38cbd32b3bd04b87a6685b4a43ea00d05aa', text='Artificial General Intelligence (AGI) 1 is an important and sometimes controversial concept in computing research, used to describe an AI system that is at least as capable as a human at most tasks. Given the rapid advancement of Machine Learning (ML) models, the concept of AGI has passed from being the subject of philosophical debate to one with near-term practical relevance. Some exp

In [4]:
from typing import Any, List

import torch
from llama_index import Document

from llama_index.embeddings import HuggingFaceEmbedding
from src.model.llama_cpp import create_llama_cpp_model
from src.translator.llamaindex_summarizer import LlamaIndexSummarizer

device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [5]:
llm_model = create_llama_cpp_model(
    package_name="llama_index",
    model_path="/home/paper_translator/data/models/ELYZA-japanese-Llama-2-7b-fast-instruct-q4_K_M.gguf",
    temperature=0.1,
)

ggml_init_cublas: found 1 CUDA devices:
  Device 0: NVIDIA GeForce GTX 1660 Ti, compute capability 7.5
llama_model_loader: loaded meta data with 21 key-value pairs and 291 tensors from /home/paper_translator/data/models/ELYZA-japanese-Llama-2-7b-fast-instruct-q4_K_M.gguf (version GGUF V2 (latest))
llama_model_loader: - tensor    0:                token_embd.weight q4_K     [  4096, 45043,     1,     1 ]
llama_model_loader: - tensor    1:              blk.0.attn_q.weight q4_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    2:              blk.0.attn_k.weight q4_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    3:              blk.0.attn_v.weight q6_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    4:         blk.0.attn_output.weight q4_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_gate.weight q4_K     [  4096, 11008,     1,     1 ]
llama_model_loader: - tensor    6:            blk.0.f

In [6]:
def _create_huggingface_embeddings(
    model_name: str, max_length: int = 512, device: torch.device = "cpu"
) -> Any:
    """HuggingFaceEmbeddingsのモデルを作成する関数

    Args:
        model_name (str): モデル名
        max_length (int, optional): 最大トークン数. Defaults to 512.
        device (torch.device, optional): デバイス. Defaults to "cpu".

    Returns:
        embed_model (Any): HuggingFaceEmbeddingsのモデル
    """
    embed_model = HuggingFaceEmbedding(
        model_name=model_name, max_length=max_length, device=device
    )
    return embed_model

In [7]:
model_name = "sentence-transformers/all-MiniLM-l6-v2"
embed_model = _create_huggingface_embeddings(
    model_name=model_name, max_length=3072, device=device
)

In [8]:
summarizer = LlamaIndexSummarizer(
    llm_model=llm_model,
    embed_model=embed_model,
    node_parser="sentence",
    is_debug=False,
)

In [9]:
def create_doc_summary_index(documents: List[Document], summarizer: Any) -> Any:
    """doc_summary_indexを作成する関数

    Args:
        documents (List[Document]): LlamaIndexのDocumentリスト
        summarizer (Any): llamaindex_summaryzer

    Returns:
        doc_summary_index (Any): doc_summary_index
    """
    doc_summary_index = summarizer.from_documents(documents=documents)
    return doc_summary_index

doc_summary_index = create_doc_summary_index(documents, summarizer)

current doc id: 0
current doc id: 1
current doc id: 2


In [10]:
translated_text = doc_summary_index.get_document_summary("1")
translated_text

'このテキストは、AGI（Artificial General Intelligence：人工汎用知能）の定義についての議論をまとめたものです。AGIは、人間のような幅広いタスクを実行できる知能を持つ人工知能のことを指します。テキストでは、9つの代表的なAGIの定義が紹介されており、それぞれの定義の強みと限界について考察されています。\n\n例えば、チューリングテストは、機械が人間と区別できるかどうかを判断するためのテストですが、現代の言語モデルはこのテストに合格することができるため、AGIを評価するための十分な基準ではないとされています。また、強いAIや人間の脳に類似したアナロジーなど、他の定義も議論されていますが、それぞれには限定的な視点や問題点が存在します。\n\nさらに、経済的な価値や柔軟性、学習能力などを含めた定義もありますが、それぞれには異なる視点や評価基準があります。また、現在の言語モデルが既にAGIと見なされることも提案されていますが、その場合でも性能の面での課題が指摘されています。\n\n総じて、AGIの定義にはさまざまな視点や基準が存在し、それぞれには利点と欠点があります。AGIの定義については、能力や性能、柔軟性、学習能力などの要素を総合的に考慮する必要があります。'

In [11]:
print(f"{translated_text}" in "、")

False


In [12]:
translated_text = translated_text.split("、", 1)[1::][0]
translated_text

'AGI（Artificial General Intelligence：人工汎用知能）の定義についての議論をまとめたものです。AGIは、人間のような幅広いタスクを実行できる知能を持つ人工知能のことを指します。テキストでは、9つの代表的なAGIの定義が紹介されており、それぞれの定義の強みと限界について考察されています。\n\n例えば、チューリングテストは、機械が人間と区別できるかどうかを判断するためのテストですが、現代の言語モデルはこのテストに合格することができるため、AGIを評価するための十分な基準ではないとされています。また、強いAIや人間の脳に類似したアナロジーなど、他の定義も議論されていますが、それぞれには限定的な視点や問題点が存在します。\n\nさらに、経済的な価値や柔軟性、学習能力などを含めた定義もありますが、それぞれには異なる視点や評価基準があります。また、現在の言語モデルが既にAGIと見なされることも提案されていますが、その場合でも性能の面での課題が指摘されています。\n\n総じて、AGIの定義にはさまざまな視点や基準が存在し、それぞれには利点と欠点があります。AGIの定義については、能力や性能、柔軟性、学習能力などの要素を総合的に考慮する必要があります。'